Importing Libraries

In [9]:
import warnings
# Suppress specific warning related to Keras input structure from FER library
warnings.filterwarnings("ignore", message="The structure of `inputs` doesn't match the expected structure.")

import cv2
import mediapipe as mp
import numpy as np
from collections import deque
from fer import FER

Helping function for drawing of lines and shadows

In [17]:
# Draw text with a white shadow behind for readability on any background
def draw_text_with_shadow(image, text, pos, font_scale, color, thickness):
    x, y = pos
    cv2.putText(image, text, (x + 1, y + 1), cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, (255, 255, 255), thickness + 1, cv2.LINE_AA)
    cv2.putText(image, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, thickness, cv2.LINE_AA)

Function for checking the hand opening

In [18]:
# Determine if hand is open by counting raised fingers and checking finger spread angle
def is_hand_open(hand_landmarks, image_width, image_height):
    wrist = hand_landmarks.landmark[0]
    tips = [hand_landmarks.landmark[i] for i in [4, 8, 12, 16, 20]]
    wrist_y = wrist.y

    # Count fingers raised above wrist vertically (exclude thumb at tips[0])
    count_up = sum(lm.y < wrist_y for lm in tips[1:])
    
    # Calculate spread angle between index and pinky fingertips
    idx_tip = tips[1]
    pinky_tip = tips[4]
    idx_x, idx_y = int(idx_tip.x * image_width), int(idx_tip.y * image_height)
    pinky_x, pinky_y = int(pinky_tip.x * image_width), int(pinky_tip.y * image_height)
    wrist_x, wrist_y_px = int(wrist.x * image_width), int(wrist.y * image_height)

    vec_idx = np.array([idx_x - wrist_x, idx_y - wrist_y_px])
    vec_pinky = np.array([pinky_x - wrist_x, pinky_y - wrist_y_px])

    vec_idx_norm = vec_idx / (np.linalg.norm(vec_idx) + 1e-6)
    vec_pinky_norm = vec_pinky / (np.linalg.norm(vec_pinky) + 1e-6)
    angle_rad = np.arccos(np.clip(np.dot(vec_idx_norm, vec_pinky_norm), -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    # Hand is open if at least 4 fingers raised and spread angle > 30 degrees
    return count_up >= 4 and angle_deg > 30

Initializing Mediapipe utilities

In [19]:
# Initialize MediaPipe holistic and drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

Detection of facial landmarks

In [20]:
# Facial landmarks to label
important_face_landmarks = {
    "Forehead": 10, "Eyes": 33, "Nose": 1, "Lips": 61,
    "Chin": 199, "Left Ear": 127, "Right Ear": 356
}

# Body landmarks to label and assess
important_body_landmarks = {
    "Left Shoulder": 11, "Right Shoulder": 12,
    "Left Elbow": 13, "Right Elbow": 14,
    "Left Wrist": 15, "Right Wrist": 16,
    "Left Hip": 23, "Right Hip": 24,
    "Left Knee": 25, "Right Knee": 26,
    "Left Ankle": 27, "Right Ankle": 28
}

# Finger landmarks mapping to names
finger_landmarks = {
    0: "Palm", 4: "Thumb Tip", 8: "Index Tip",
    12: "Middle Tip", 16: "Ring Tip", 20: "Pinky Tip"
}

# Offsets for text labeling
offsets = {
    "Left": (-25, -12),
    "Right": (23, -10),
    "default": (10, -14)
}

Styling part

In [21]:
# Drawing parameters
line_thickness = 1
circle_radius = 2
font_scale = 0.35
text_thickness = 1

Emotion Tracking

In [22]:
# Initialize FER facial emotion detector with MTCNN for better accuracy
emotion_detector = FER(mtcnn=True)

# For fingertip trails visualization
max_trails_len = 20
finger_trails_right = {i: deque(maxlen=max_trails_len) for i in finger_landmarks.keys()}
finger_trails_left = {i: deque(maxlen=max_trails_len) for i in finger_landmarks.keys()}

# For smoothing emotion output to reduce jitter
emotion_history = deque(maxlen=10)
def smooth_emotion(emotion_label):
    emotion_history.append(emotion_label)
    return max(set(emotion_history), key=emotion_history.count)

# Custom heuristic for sleepy/active state based on eye openness landmarks
def detect_sleepy_active(face_landmarks, width, height):
    try:
        left_eye_upper = face_landmarks.landmark[159]
        left_eye_lower = face_landmarks.landmark[145]
        right_eye_upper = face_landmarks.landmark[386]
        right_eye_lower = face_landmarks.landmark[374]

        left_eye_oen = abs(left_eye_upper.y - left_eye_lower.y)
        right_eye_open = abs(right_eye_upper.y - right_eye_lower.y)
        avg_eye_open = (left_eye_open + right_eye_open) / 2

        sleepy_threshold = 0.008
        active_threshold = 0.02

        if avg_eye_open < sleepy_threshold:
            return "Sleepy"
        elif avg_eye_open > active_threshold:
            return "Active"
        else:
            return None
    except Exception:
        return None

Setting up the webcam (upto full screen)

In [23]:
# Setup webcam feed and fullscreen window
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)   # Reduce resolution for speed (adjust as needed)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

window_name = 'MediaPipe + FER Emotion Detection'
cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

paused = False
img_count = 0
theme_idx = 0

# Color themes for landmark drawing
color_themes = [
    {'dot': (0, 0, 255), 'line': (0, 0, 0)},         # Red dots, black lines
    {'dot': (0, 255, 0), 'line': (160, 32, 240)},   # Green dots, purple lines
    {'dot': (255, 127, 36), 'line': (30, 30, 30)}   # Orange dots, dark gray lines
]

# Frame counter for skipping emotion analysis frames to improve speed
emotion_frame_skip = 5
frame_counter = 0
cached_emotions = []
cached_emotion_label = "Neutral"

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to RGB for MediaPipe
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image_rgb)

            # Convert back to BGR for OpenCV display
            image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
            h, w, _ = image.shape
            color_theme = color_themes[theme_idx]

            # Draw and label face landmarks
            if results.face_landmarks:
                for name, idx in important_face_landmarks.items():
                    try:
                        lm = results.face_landmarks.landmark[idx]
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(image, (x, y), circle_radius, color_theme['dot'], -1, cv2.LINE_AA)
                        dx, dy = offsets["default"]
                        draw_text_with_shadow(image, name, (x + dx, y + dy), font_scale, (0, 0, 0), text_thickness)
                    except Exception:
                        pass

            # Draw and label body landmarks
            if results.pose_landmarks:
                for name, idx in important_body_landmarks.items():
                    try:
                        lm = results.pose_landmarks.landmark[idx]
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(image, (x, y), circle_radius + 1, color_theme['dot'], -1, cv2.LINE_AA)
                        if "Left" in name:
                            dx, dy = offsets["Left"]
                        elif "Right" in name:
                            dx, dy = offsets["Right"]
                        else:
                            dx, dy = offsets["default"]
                        draw_text_with_shadow(image, name, (x + dx, y + dy), font_scale, (0, 0, 0), text_thickness)
                    except Exception:
                        pass

                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_holistic.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness, circle_radius=circle_radius),
                    mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness)
                )

            # Process hands with fingertip trails and open hand detection messages
            hands_open_messages = []
            for hand_side, hand_landmarks, finger_trails in [
                ("Right", results.right_hand_landmarks, finger_trails_right),
                ("Left", results.left_hand_landmarks, finger_trails_left)
            ]:
                if hand_landmarks:
                    # Draw fingertips and label them
                    for idx, part in finger_landmarks.items():
                        try:
                            lm = hand_landmarks.landmark[idx]
                            x, y = int(lm.x * w), int(lm.y * h)
                            finger_trails[idx].append((x, y))
                            cv2.circle(image, (x, y), circle_radius + 1, color_theme['dot'], -1, cv2.LINE_AA)
                            dx, dy = offsets[hand_side]
                            draw_text_with_shadow(image, f"{hand_side} {part}", (x + dx, y + dy), font_scale, (0, 0, 0), text_thickness)
                        except Exception:
                            pass

                    # Draw hand skeleton connections
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_holistic.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness, circle_radius=circle_radius),
                        mp_drawing.DrawingSpec(color=color_theme['line'], thickness=line_thickness)
                    )

                    # If hand is open, add a message (yellow wrist-to-fingertip lines removed)
                    if is_hand_open(hand_landmarks, w, h):
                        hands_open_messages.append(f"{hand_side} Hand Open")

                    # Draw fingertip trails as lines
                    for trail in finger_trails.values():
                        for i in range(1, len(trail)):
                            cv2.line(image, trail[i - 1], trail[i], (0, 255, 255), 1)

            # Display hand open messages with spacing
            base_y = 140
            for i, msg in enumerate(hands_open_messages):
                draw_text_with_shadow(image, msg, (15, base_y + i * 30), 0.7, (0, 255, 0), 2)

            # If both hands detected, calculate and display approximate wrist distance in meters
            if results.left_hand_landmarks and results.right_hand_landmarks:
                left_wrist = results.left_hand_landmarks.landmark[0]
                right_wrist = results.right_hand_landmarks.landmark[0]

                lw = np.array([left_wrist.x, left_wrist.y, left_wrist.z])
                rw = np.array([right_wrist.x, right_wrist.y, right_wrist.z])

                dist_norm = np.linalg.norm(lw - rw)
                scale_factor = 2.0  # Scale factor for approximate meters conversion
                distance_meters = dist_norm * scale_factor

                dist_text = f"Hands Distance: {distance_meters:.2f} m"
                draw_text_with_shadow(image, dist_text, (15, base_y + len(hands_open_messages) * 30), 0.7, (255, 255, 0), 2)

            # Update frame counter, run emotion inference every N frames for efficiency
            frame_counter += 1
            if frame_counter % emotion_frame_skip == 0:
                rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                cached_emotions = emotion_detector.detect_emotions(rgb_frame)

            # Detect sleepy or active custom emotion
            custom_emotion = None
            if results.face_landmarks:
                custom_emotion = detect_sleepy_active(results.face_landmarks, w, h)

            # Extract top FER emotion or use neutral
            if cached_emotions:
                top_emotion = max(cached_emotions[0]['emotions'].items(), key=lambda x: x[1])
                raw_emotion = f"{top_emotion[0].capitalize()} ({top_emotion[1] * 100:.1f}%)"
            else:
                raw_emotion = "Neutral"

            # Smooth emotion output or show custom state
            cached_emotion_label = custom_emotion if custom_emotion else smooth_emotion(raw_emotion)

            # Display emotion on screen
            draw_text_with_shadow(image, cached_emotion_label, (15, 80), 0.7, (0, 0, 255), 2)

            # Pose confidence and posture metrics
            if results.pose_landmarks:
                torso_ids = [11, 12, 23, 24, 25, 26]
                visibilities = [results.pose_landmarks.landmark[i].visibility for i in torso_ids]
                avg_conf = sum(visibilities) / len(visibilities)
                confidence_str = f"Pose Confidence: {avg_conf:.2f}"

                left_shoulder = results.pose_landmarks.landmark[11]
                left_hip = results.pose_landmarks.landmark[23]
                torso_vec = np.array([left_shoulder.x - left_hip.x, left_shoulder.y - left_hip.y])
                vertical_vec = np.array([0, -1])
                torso_vec_norm = torso_vec / (np.linalg.norm(torso_vec) + 1e-6)
                angle_rad = np.arccos(np.clip(np.dot(torso_vec_norm, vertical_vec), -1.0, 1.0))
                angle_deg = np.degrees(angle_rad)
                posture_str = "Good Posture" if angle_deg < 15 else "Poor Posture"

                cv2.putText(image, confidence_str, (15, 40), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0) if avg_conf > 0.5 else (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(image, posture_str, (15, 110), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0) if posture_str == "Good Posture" else (0, 0, 255), 2, cv2.LINE_AA)

            # Display control instructions
            info_text = "q=quit | space=pause/resume | s=screenshot | c=theme"
            cv2.putText(image, info_text, (8, h - 16), cv2.FONT_HERSHEY_SIMPLEX,
                        0.38, (80, 40, 10), 1, cv2.LINE_AA)

            # Show final frame
            cv2.imshow(window_name, image)

        # Handle keyboard input regardless of pause state
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break
        elif key == ord(' '):
            paused = not paused
        elif key == ord('s') and not paused:
            img_count += 1
            cv2.imwrite(f"screenshot_{img_count}.png", image)
            print(f"Screenshot saved as screenshot_{img_count}.png")
        elif key == ord('c'):
            theme_idx = (theme_idx + 1) % len(color_themes)

# Cleanup
cap.release()
cv2.destroyAllWindows()